In [132]:
import requests
from bs4 import BeautifulSoup
import lxml
import csv

In [133]:
url = 'https://move.ru/kvartiry/'

In [134]:
response = requests.get(url)

In [135]:
soup = BeautifulSoup(response.text, 'lxml')

Получить все ссылки на объявления со страницы

In [136]:
def get_links(soup):    
    links = []
    items = soup.find_all('div', class_='search-item__header')
    for item in items:
        link = item.find('a', class_='search-item__title-link search-item__item-link')
        if link:
            href = link.get('href')
            full_link = f"https:{href}" 
            links.append(full_link)
    return links

Получить значение признака по имени

In [137]:
def get_value(property_name, soup):
    property_names = soup.find_all('div', class_='object-info__details-table_property_name')

    for name in property_names:
        if property_name in name.text:
            value = name.find_next_sibling('div', class_='object-info__details-table_property_value')
            if value:
                value_text = value.text.strip()
                return value_text
    return '?'

Получить все значения признаков

In [138]:
def get_all_values(soup):    
    price = get_value("Цена", soup).replace('₽', '').replace(' ', '')
    price_per_m2 = get_value("цена за м²", soup).replace('₽', '').replace(' ', '')
    level, count_levels = (get_value("Этаж", soup).split('/') + ['?'])[:2]
    ceiling_height = get_value("Высота потолков", soup).replace('м', '').strip()
    building_type = get_value("Тип объекта", soup)
    advertisement_type = get_value("Тип объявления", soup)
    total_space = get_value("Общая площадь", soup).replace('м²', '').strip()
    living_space = get_value("Жилая площадь", soup).replace('м²', '').strip()
    kitchen_space = get_value("Площадь кухни", soup).replace('м²', '').strip()

    return [price, price_per_m2, level, count_levels, ceiling_height, building_type, advertisement_type, total_space, living_space, kitchen_space]

In [139]:
def get_all_values_from_page(url, writer):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    links = get_links(soup)
    for link in links:
        try:
            response_element_page = requests.get(link)
            soup_element_page = BeautifulSoup(response_element_page.text, 'lxml')
            values = get_all_values(soup_element_page)
            writer.writerow(values)
        except (requests.exceptions.RequestException, Exception) as e:
            print("too long")

In [140]:
with open('data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    writer.writerow(['Price', 'Price for square meter', 'Level', 'Levels count', 'Ceiling height', 'Building type', 'Advertisement type', 'Total space', 'Living space', 'Kitchen space'])
    for i in range(1, 15):
      get_all_values_from_page(url+"/?page={i}&limit=100", writer)

kaka
kaka
